In [1]:
!pip install -q --disable-pip-version-check yamlmagic
%load_ext yamlmagic

In [2]:
%%yaml model
id: https://example.org/
name: example
prefixes:
  example: https://example.org/
  linkml: https://w3id.org/linkml/
default_prefix: example
default_range: string
imports:
  - linkml:types

classes:
  Thing:
    attributes:
      type:
        designates_type: true
  Vehicle:
    is_a: Thing
    attributes:
      num_wheels:
        range: integer
  Building:
    is_a: Thing
    attributes:
      num_floors:
        range: integer
        
  Container:
    attributes:
      things:
        range: Thing
        multivalued: true
    

<IPython.core.display.Javascript object>

In [3]:
import yaml
model_str = yaml.dump(model)

In [4]:
from linkml_runtime.utils.schemaview import SchemaView

In [5]:
sv = SchemaView(model_str)
schema = sv.schema

In [6]:
from linkml.generators.pydanticgen import PydanticGenerator
import yaml

In [7]:
import logging
gen = PydanticGenerator(schema, log_level=logging.ERROR)

In [8]:
print(gen.serialize())

from __future__ import annotations 

import re
import sys
from datetime import (
    date,
    datetime,
    time
)
from decimal import Decimal 
from enum import Enum 
from typing import (
    Any,
    ClassVar,
    Dict,
    List,
    Literal,
    Optional,
    Union
)

from pydantic import (
    BaseModel,
    ConfigDict,
    Field,
    RootModel,
    field_validator
)


metamodel_version = "None"
version = "None"


class ConfiguredBaseModel(BaseModel):
    model_config = ConfigDict(
        validate_assignment = True,
        validate_default = True,
        extra = "forbid",
        arbitrary_types_allowed = True,
        use_enum_values = True,
        strict = False,
    )
    pass




class LinkMLMeta(RootModel):
    root: Dict[str, Any] = {}
    model_config = ConfigDict(frozen=True)

    def __getattr__(self, key:str):
        return getattr(self.root, key)

    def __getitem__(self, key:str):
        return self.root[key]

    def __setitem__(self, key:str, value):
        se

In [9]:
mod = gen.compile_module()

In [10]:
mod.Container.model_validate({"things": [{"type": "Vehicle", "num_wheels": 2}]})

Container(things=[Vehicle(type='Vehicle', num_wheels=2)])

In [11]:
import pytest

In [12]:
with pytest.raises(ValueError):
  mod.Container.model_validate({"things": [{"type": "Vehicle", "num_floors": 4}]})

In [13]:
mod.Container(things=[mod.Vehicle(num_wheels=2)])

Container(things=[Vehicle(type='Vehicle', num_wheels=2)])

In [14]:
with pytest.raises(ValueError):
    mod.Container(things=[mod.Thing(type="Vehicle", num_wheels=2)])

## Change type designator range

What happens if the range is a curie?

In [15]:
schema.classes["Thing"].attributes["type"].range = "uriorcurie"
gen = PydanticGenerator(schema, log_level=logging.ERROR)
mod = gen.compile_module()
print(gen.serialize())

from __future__ import annotations 

import re
import sys
from datetime import (
    date,
    datetime,
    time
)
from decimal import Decimal 
from enum import Enum 
from typing import (
    Any,
    ClassVar,
    Dict,
    List,
    Literal,
    Optional,
    Union
)

from pydantic import (
    BaseModel,
    ConfigDict,
    Field,
    RootModel,
    field_validator
)


metamodel_version = "None"
version = "None"


class ConfiguredBaseModel(BaseModel):
    model_config = ConfigDict(
        validate_assignment = True,
        validate_default = True,
        extra = "forbid",
        arbitrary_types_allowed = True,
        use_enum_values = True,
        strict = False,
    )
    pass




class LinkMLMeta(RootModel):
    root: Dict[str, Any] = {}
    model_config = ConfigDict(frozen=True)

    def __getattr__(self, key:str):
        return getattr(self.root, key)

    def __getitem__(self, key:str):
        return self.root[key]

    def __setitem__(self, key:str, value):
        se

In [16]:
mod.Container.model_validate({"things": [{"type": "example:Vehicle", "num_wheels": 2}]})

Container(things=[Vehicle(type='example:Vehicle', num_wheels=2)])

In [17]:
schema.classes["Thing"].attributes["type"].range = "uri"
gen = PydanticGenerator(schema, log_level=logging.ERROR)
mod = gen.compile_module()
print(gen.serialize())

from __future__ import annotations 

import re
import sys
from datetime import (
    date,
    datetime,
    time
)
from decimal import Decimal 
from enum import Enum 
from typing import (
    Any,
    ClassVar,
    Dict,
    List,
    Literal,
    Optional,
    Union
)

from pydantic import (
    BaseModel,
    ConfigDict,
    Field,
    RootModel,
    field_validator
)


metamodel_version = "None"
version = "None"


class ConfiguredBaseModel(BaseModel):
    model_config = ConfigDict(
        validate_assignment = True,
        validate_default = True,
        extra = "forbid",
        arbitrary_types_allowed = True,
        use_enum_values = True,
        strict = False,
    )
    pass




class LinkMLMeta(RootModel):
    root: Dict[str, Any] = {}
    model_config = ConfigDict(frozen=True)

    def __getattr__(self, key:str):
        return getattr(self.root, key)

    def __getitem__(self, key:str):
        return self.root[key]

    def __setitem__(self, key:str, value):
        se

## Data Classes

## Any Of

In [18]:
%%yaml model
id: https://example.org/
name: example
prefixes:
  example: https://example.org/
  linkml: https://w3id.org/linkml/
default_prefix: example
imports:
  - linkml:types

classes:
  Thing:
    attributes:
      type:
        designates_type: true
        range: string
  Vehicle:
    is_a: Thing
    attributes:
      num_wheels:
        range: integer
  Car:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "2"
  Bicycle:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "4"
  Building:
    is_a: Thing
    attributes:
      num_floors:
        range: integer
        
  Container:
    attributes:
      things:
        any_of:
          - range: Vehicle
          - range: Building
        multivalued: true
    

<IPython.core.display.Javascript object>

In [19]:
model_str = yaml.dump(model)
sv = SchemaView(model_str)
schema = sv.schema

In [20]:
from logging import ERROR
gen = PydanticGenerator(schema, log_level=ERROR)
mod = gen.compile_module()
print(gen.serialize())

from __future__ import annotations 

import re
import sys
from datetime import (
    date,
    datetime,
    time
)
from decimal import Decimal 
from enum import Enum 
from typing import (
    Any,
    ClassVar,
    Dict,
    List,
    Literal,
    Optional,
    Union
)

from pydantic import (
    BaseModel,
    ConfigDict,
    Field,
    RootModel,
    field_validator
)


metamodel_version = "None"
version = "None"


class ConfiguredBaseModel(BaseModel):
    model_config = ConfigDict(
        validate_assignment = True,
        validate_default = True,
        extra = "forbid",
        arbitrary_types_allowed = True,
        use_enum_values = True,
        strict = False,
    )
    pass




class LinkMLMeta(RootModel):
    root: Dict[str, Any] = {}
    model_config = ConfigDict(frozen=True)

    def __getattr__(self, key:str):
        return getattr(self.root, key)

    def __getitem__(self, key:str):
        return self.root[key]

    def __setitem__(self, key:str, value):
        se

In [21]:
# TODO: flatten unions

In [22]:
mod.Container.model_validate({"things": [{"type": "Vehicle"}]})

Container(things=[Vehicle(type='Vehicle', num_wheels=None)])

In [23]:
# TODO: expressions

## Union Of

In [24]:
%%yaml model
id: https://example.org/
name: example
prefixes:
  example: https://example.org/
  linkml: https://w3id.org/linkml/
default_prefix: example
imports:
  - linkml:types

classes:
  Thing:
    attributes:
      type:
        designates_type: true
        range: string
  Vehicle:
    is_a: Thing
    attributes:
      num_wheels:
        range: integer
  Car:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "2"
  Bicycle:
    is_a: Vehicle
    slot_usage:
      num_wheels:
        equals_expression: "4"
  Building:
    is_a: Thing
    attributes:
      num_floors:
        range: integer
        
  Container:
    attributes:
      things:
        any_of:
          - range: Vehicle
          - range: Building
        multivalued: true
    

<IPython.core.display.Javascript object>